In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import IntegerType
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

# Obter as chaves da AWS das variáveis de ambiente
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')


# Iniciando uma SparkSession com Delta Lake
spark = SparkSession.builder \
    .appName("SilverLayer") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Adicionando configurações S3 diretamente na sessão Spark
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key_id)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_access_key)
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.amazonaws.com")
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

# Configurações do bucket S3
silver_bucket = "s3a://engenharia-dados-satc-silver-bucket"
gold_bucket = "s3a://engenharia-dados-satc-gold-bucket"

# Carregar arquivos Delta da camada silver
datas = spark.read.format("delta").load(f'{silver_bucket}/dim_data')
pedidos = spark.read.format("delta").load(f'{silver_bucket}/pedidos')
localizacoes = spark.read.format("delta").load(f'{silver_bucket}/localizacoes')


In [3]:
# Renomear colunas e adicionar IDs sequenciais
pedidos = pedidos.withColumn('sk_pedido', col('idpedido').cast(IntegerType()))
localizacoes = localizacoes.withColumn('sk_loc', col('cep').cast(IntegerType()))

# Calcular volume e massa dos pedidos
pedidos = pedidos.withColumn('volume', (col('comprimento_cm') * col('altura_cm') * col('largura_cm')) / (100*100*100))
pedidos = pedidos.withColumn('massa', col('peso_g') / 1000)

In [4]:
# Selecionar colunas para transações
transacoes = pedidos.select(
    col('sk_pedido').alias('sk_pedido'),
    col('idcliente').alias('idcliente'),
    col('idvendedor').alias('idvendedor'),
    col('pagamento').alias('pagamento'),
    col('parcelas').alias('parcelas')
)

### CRIANDO ENTREGAS ###
entregas = pedidos.select(
    col('sk_pedido').alias('sk_pedido'),
    col('categoria').alias('categoria'),
    col('statuspedido').alias('statuspedido'),
    col('cepcliente').alias('cepcliente'),
    col('cepvendedor').alias('cepvendedor')
)
entregas = entregas.join(localizacoes.withColumnRenamed('sk_loc', 'destino'), entregas.cepcliente == localizacoes.cep, 'left')

entregas = entregas.select(
    col('sk_pedido').alias('sk_pedido'),
    col('categoria').alias('categoria'),
    col('statuspedido').alias('statuspedido'),
    col('destino').alias('destino'),
    col('cepvendedor').alias('cepvendedor')
)

entregas = entregas.join(localizacoes.withColumnRenamed('sk_loc', 'origem'), entregas.cepvendedor == localizacoes.cep, 'left')

entregas = entregas.drop('cepvendedor')


In [5]:
### CRIANDO PEDIDOS ###

pedidos = pedidos.withColumn('datacompra', to_date(col('datacompra')))
pedidos = pedidos.withColumn('datacliente', to_date(col('datacliente')))
pedidos = pedidos.withColumn('prazo', to_date(col('prazo')))

pedidos = pedidos.withColumn('ped_diasentrega', (col('datacliente').cast('long') - col('datacompra').cast('long')) / (24*60*60))
pedidos = pedidos.withColumn('ped_diasprazo', (col('prazo').cast('long') - col('datacompra').cast('long')) / (24*60*60))

fato = pedidos.select(
    col('sk_pedido').alias('sk_pedido'),
    col('datacompra').alias('datacompra'),
    col('iditem').alias('iditem'),
    col('preco').alias('preco'),
    col('frete').alias('frete'),
    col('volume').alias('volume'),
    col('massa').alias('massa'),
    col('ped_diasentrega').alias('ped_diasentrega'),
    col('ped_diasprazo').alias('ped_diasprazo')
)

datas = datas.withColumn('dt_data', to_date(col('dt_data')))
fato = fato.join(datas, fato.datacompra == datas.dt_data, 'left').select(
    col('sk_pedido').alias('sk_pedido'),
    col('sk_data').alias('sk_data'),
    col('iditem').alias('iditem'),
    col('preco').alias('preco'),
    col('frete').alias('frete'),
    col('volume').alias('volume'),
    col('massa').alias('massa'),
    col('ped_diasentrega').alias('ped_diasentrega'),
    col('ped_diasprazo').alias('ped_diasprazo')
)

C:\Users\henri\AppData\Local\Temp\ipykernel_6324\2745140035.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dim_transacao['sk_cliente'] = range(1, len(dim_transacao) + 1)


In [6]:
### SALVAR TABELAS NA CAMADA GOLD ###

datas.write.format("delta").mode('overwrite').save(f'{gold_bucket}/dim_data')
fato.write.format("delta").mode('overwrite').save(f'{gold_bucket}/fato_pedidos')
entregas.write.format("delta").mode('overwrite').save(f'{gold_bucket}/dim_entregas')
transacoes.write.format("delta").mode('overwrite').save(f'{gold_bucket}/dim_transacao')
localizacoes.write.format("delta").mode('overwrite').save(f'{gold_bucket}/dim_localizacoes')